In [1]:
from keras.datasets import cifar10  
from keras.utils import np_utils  
from keras.models import Sequential  
from keras.layers.core import Dense, Dropout, Activation, Flatten 
from keras.layers.convolutional import Conv2D, MaxPooling2D  
from keras.optimizers import SGD, Adam, RMSprop  
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt  
import numpy  as np   

# For solving CERTIFICATE_VERIFY_FAILED error when downloading CIFAR10
# From user Denise Mak posted on stack overflow
# url: https://stackoverflow.com/questions/69687794/unable-to-manually-load-cifar10-dataset
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels 
IMG_CHANNELS = 3 
IMG_ROWS = 32  
IMG_COLS = 32  

#constant  
BATCH_SIZE = 128  
NB_EPOCH = 20  
NB_CLASSES = 10  
VERBOSE = 1  
VALIDATION_SPLIT = 0.2  
OPTIM = RMSprop()  
NUM_TO_AUGMENT=5

#load dataset  
(X_train, y_train), (X_test, y_test) = cifar10.load_data()  
print('X_train shape:', X_train.shape)  
print(X_train.shape[0], 'train samples')  
print(X_test.shape[0], 'test samples') 

# augumenting 
print("Augmenting training set images...")
datagen = ImageDataGenerator( 
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
    fill_mode='nearest')

xtas, ytas = [], [] 
for i in range(X_train.shape[0]): 
    num_aug = 0 
    if (i % 2500) == 0:
        print(round((i / X_train.shape[0]) * 100), '% complete')
    x = X_train[i] # (3, 32, 32) 
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32) 
    
    for x_aug in datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='cifar', save_format='jpeg'): 
        if num_aug >= NUM_TO_AUGMENT: 
            break 
        xtas.append(x_aug[0]) 
        num_aug += 1

# one-hot encoding and normalize the images 
#
# convert to categorical  
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)  
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)  

# float and normalization  
X_train = X_train.astype('float32')  
X_test = X_test.astype('float32')  
X_train /= 255  
X_test /= 255 

# net will learn 32 convolutional filters, each with a 3x3 size. 
# max-pooling operation with pool size 2x2 and a dropout at 25% 
#
# network 
model = Sequential()  
model.add(Conv2D(32, (3, 3), padding='same',  
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))  
model.add(Activation('relu'))  
model.add(Conv2D(32, (3, 3), padding='same'))  
model.add(Activation('relu'))  
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.25))  
model.add(Conv2D(64, (3, 3), padding='same'))  
model.add(Activation('relu'))  
model.add(Conv2D(64, (3, 3), padding='same'))    
model.add(Activation('relu'))  
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.25))  
model.add(Flatten())  
model.add(Dense(512)) 
model.add(Activation('relu'))  
model.add(Dropout(0.5))  
model.add(Dense(NB_CLASSES))  
model.add(Activation('softmax')) 
model.summary() 

# After defining the network, train the model.  
# Split the data into validation, training, and testing sets.  
# Training is used to build model, validation is used to select 
# the best performing approach, the test set is to check the performance 
# of the best model on fresh unseen data 
#


Using TensorFlow backend.


170500096/170498071 [==============================] - 2s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Augmenting training set images...
0 % complete
5 % complete
10 % complete
15 % complete
20 % complete
25 % complete
30 % complete
35 % complete
40 % complete
45 % complete
50 % complete
55 % complete
60 % complete
65 % complete
70 % complete
75 % complete
80 % complete
85 % complete
90 % complete
95 % complete
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
________________________________________________________________

In [ ]:
#fit the dataget 
datagen.fit(X_train)

model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

#checkpoints
checkpoint_filepath = 'keras_check'
cp_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# train  
history = model.fit_generator(datagen.flow(X_train, Y_train, 
                                           batch_size=BATCH_SIZE), 
                              samples_per_epoch=X_train.shape[0], 
                              epochs=NB_EPOCH, 
                              verbose=VERBOSE,
                              callbacks = [cp_callback]) 
score = model.evaluate(X_test, Y_test, 
                       batch_size=BATCH_SIZE, 
                       verbose=VERBOSE) 
print("Test score:", score[0]) 
print('Test accuracy:', score[1]) 

# Save the architecture of the deep network 
#
#save model  
model_json = model.to_json()  
open('cifar10_architecture.json', 'w').write(model_json)  
# And the weights learned by our deep network on the training 
model.save_weights('cifar10_weights.h5', overwrite=True) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=20, verbose=1, callbacks=[<keras.ca..., steps_per_epoch=390)`


Epoch 1/20
390/390 [==============================] - 115s 295ms/step - loss: 1.9462 - accuracy: 0.2913
Epoch 2/20


C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


390/390 [==============================] - 116s 298ms/step - loss: 1.6666 - accuracy: 0.3971
Epoch 3/20
390/390 [==============================] - 118s 301ms/step - loss: 1.5446 - accuracy: 0.4441
Epoch 4/20
390/390 [==============================] - 113s 290ms/step - loss: 1.4533 - accuracy: 0.4768
Epoch 5/20
390/390 [==============================] - 120s 307ms/step - loss: 1.3928 - accuracy: 0.5024
Epoch 6/20
390/390 [==============================] - 114s 292ms/step - loss: 1.3504 - accuracy: 0.5180
Epoch 7/20
390/390 [==============================] - 114s 293ms/step - loss: 1.3116 - accuracy: 0.5329
Epoch 8/20
390/390 [==============================] - 119s 306ms/step - loss: 1.2787 - accuracy: 0.5474
Epoch 9/20
390/390 [==============================] - 114s 293ms/step - loss: 1.2583 - accuracy: 0.5546
Epoch 10/20
390/390 [==============================] - 120s 309ms/step - loss: 1.2393 - accuracy: 0.5632
Epoch 11/20
390/390 [==============================] - 116s 297ms/step - l

There are a lot of ethical and privacy implications revolving around image recognition AI. There can be unethical bias if the data being used to train the AI is biased for or against a group of its labels. There are concerns about where the training data comes from and the ethics of using another persons work to train something you claim as yours. There are concerns on the privacy of using an AI to train by trolling through images on the internet of people that have not consented to being a part of the training data. With the model that I created, it currently does not have any of those issues since the data is public it is not subject to privacy concerns, and since the model only differentiates between things like planes and cats it does not have really any ethical concerns beyond ensuring that the data does not contain any bias like giving favor to one group over another or excluding a group entirely.

Refernces B. C. Stahl and D. Wright, "Ethics and Privacy in AI and Big Data: Implementing Responsible Research and Innovation," in IEEE Security & Privacy, vol. 16, no. 3, pp. 26-33, May/June 2018, doi: 10.1109/MSP.2018.2701164. keywords: {Artificial intelligence;Ethics;Big Data;Data protection;artificial intelligence;smart information systems;ethics;responsible research and innovation;Human Brain Project;security;AI Ethics},